Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.
Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift

In [ ]:
path = 'https://raw.githubusercontent.com/chekhovana/courses/main/machine_learning/3_unsupervised_learning/1_clustering/data/1.5_checkins.dat'
df = pd.read_csv(path, sep='|', usecols=[3, 4], skiprows=[1], na_values=' ' * 19, 
                 low_memory=False).dropna().iloc[:100000, :]
df.columns = ['latitude', 'longitude']
df.head()

,latitude,longitude
1,38.895112,-77.036366
3,33.800745,-84.410520
7,45.523452,-122.676207
9,40.764462,-111.904565
10,33.448377,-112.074037


Необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [ ]:
ms = MeanShift(bandwidth=0.1)
ms.fit(df)

MeanShift(bandwidth=0.1)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [ ]:
unique, occurs = np.unique(ms.labels_, return_counts=True)
labels = [l for l, c in zip(unique, occurs) if c > 15]
centers = [ms.cluster_centers_[i] for i in labels]

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

In [ ]:
offices = [
    [33.751277, -118.188740], 
    [25.867736, -80.324116], 
    [51.503016, -0.075479], 
    [52.378894, 4.885084],
    [39.366487, 117.036146], 
    [-33.868457, 151.205134]
]

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [ ]:
from scipy.spatial import distance

distances = distance.cdist(offices, centers)
indexes = np.argsort(distances.flatten())
indexes = (indexes % len(centers))[:20]
np.array(centers)[indexes]

array([[-3.38606304e+01,  1.51204776e+02],
       [ 5.23729640e+01,  4.89231722e+00],
       [ 2.58456723e+01, -8.03188906e+01],
       [ 5.15029913e+01, -1.25537289e-01],
       [ 3.38098780e+01, -1.18148924e+02],
       [ 2.57858124e+01, -8.02179380e+01],
       [ 2.57053497e+01, -8.02834287e+01],
       [ 2.60100982e+01, -8.01999906e+01],
       [ 3.38883253e+01, -1.18048928e+02],
       [ 3.38729860e+01, -1.18362091e+02],
       [ 3.39725748e+01, -1.18168371e+02],
       [ 2.61388438e+01, -8.03343468e+01],
       [ 3.39839359e+01, -1.18007405e+02],
       [ 2.61208627e+01, -8.01589067e+01],
       [ 3.38173064e+01, -1.17891249e+02],
       [ 3.40603976e+01, -1.18248709e+02],
       [ 3.36743027e+01, -1.17858789e+02],
       [ 2.62005846e+01, -8.02507161e+01],
       [ 3.40354870e+01, -1.18438998e+02],
       [ 3.41314601e+01, -1.18118012e+02]])